In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/voice-messages/voice_messages/audio_103@31-07-2023_09-47-11.ogg
/kaggle/input/voice-messages/voice_messages/audio_54@22-04-2023_18-50-33.ogg
/kaggle/input/voice-messages/voice_messages/audio_58@17-07-2023_13-19-34.ogg
/kaggle/input/voice-messages/voice_messages/audio_105@31-07-2023_17-51-36.ogg
/kaggle/input/voice-messages/voice_messages/audio_78@30-06-2023_19-48-49.ogg
/kaggle/input/examples-of-calls/samples/Звонок в ТехПоддержку  САРРАСАРРА ВЕЙ ИЗ МИР ШАНА ТОВА.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку КорбинаТелеком  Во дает.mp3
/kaggle/input/examples-of-calls/samples/Corbina — Звонок в техподдержку.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку MegaFon  Неслышат звонок.mp3
/kaggle/input/examples-of-calls/samples/Nightmare  Звонок в техподдержку Rambler.mp3
/kaggle/input/rings-29-9/29_9/79266440178_in_2023_09_21-14_28_09_74956044009_tmbo.mp3
/kaggle/input/rings-29-9/29_9/fetisova_inna_out_2023_09_26-15_55_10_79273659168_2kwp.m

In [2]:
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm

__import__('warnings').filterwarnings("ignore")

Автоматическое скачивание кода из репозитария на гитхабе в нужную папку

In [3]:
!wget -O sp_addons.py https://raw.githubusercontent.com/lex51/speech_processing/main/experiments/sp_addons.py

--2023-10-03 13:54:25--  https://raw.githubusercontent.com/lex51/speech_processing/main/experiments/sp_addons.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12716 (12K) [text/plain]
Saving to: ‘sp_addons.py’

sp_addons.py        100%[===================>]  12.42K  --.-KB/s    in 0s      

2023-10-03 13:54:25 (54.6 MB/s) - ‘sp_addons.py’ saved [12716/12716]



In [4]:
from sp_addons import AudioManipulation

In [5]:
from statistics import mode
from pydub import AudioSegment
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio


# Указать устройство (GPU) для вычислений, если оно доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и экстрактора признаков на устройство GPU
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForSequenceClassification.from_pretrained("xbgoose/hubert-speech-emotion-recognition-russian-dusha-finetuned")
model.to(device)

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (pro

In [6]:
# EXAMPLES = Path(r'/kaggle/input/examples-of-calls')
# EXAMPLES = Path(r'/kaggle/input/voice-messages')
EXAMPLES = Path(r'/kaggle/input/rings-29-9')

WORK_PATH = Path('.')

filenames = [EXAMPLES.joinpath(file) for file in glob(f'{EXAMPLES}/**', recursive=True)
             if Path(file).is_file()]
device

device(type='cuda')

In [7]:
filenames[:5]

[PosixPath('/kaggle/input/rings-29-9/29_9/79266440178_in_2023_09_21-14_28_09_74956044009_tmbo.mp3'),
 PosixPath('/kaggle/input/rings-29-9/29_9/fetisova_inna_out_2023_09_26-15_55_10_79273659168_2kwp.mp3'),
 PosixPath('/kaggle/input/rings-29-9/29_9/mikhaylova_polina_out_2023_09_29-10_16_35_79095082928_p4j6.mp3'),
 PosixPath('/kaggle/input/rings-29-9/29_9/79260007055_in_2023_09_26-12_32_39_79266662497_ko7b.mp3'),
 PosixPath('/kaggle/input/rings-29-9/29_9/morozova_anastasia_out_2023_09_12-11_04_57_79255070348_7mfb.mp3')]

In [8]:
audio_obj = AudioManipulation(feature_extractor=feature_extractor, model=model)
audio_obj.device

device(type='cuda')

In [9]:
%%time
name_dataset = str(EXAMPLES).split('/')[-1].replace('-','_')
df_preds = pd.DataFrame(columns=['file_name', 'emotion', 'all_emotions'])
for file_path in tqdm(filenames[:]):
    try:
        predicts = audio_obj.predict(file_path, simple_indexes=False, show_file_info=True)
    except:
        predicts = ('', ['ошибка', 'не получилось обработать файл'])
    df_preds.loc[len(df_preds)] = [Path(file_path).name, *predicts]
    print('Эмоция:', predicts[0])
    
df_preds.to_csv(f'df_preds_{name_dataset}.csv', sep=';', index=False)

  0%|          | 0/19 [00:00<?, ?it/s]

Файл: 79266440178_in_2023_09_21-14_28_09_74956044009_tmbo.mp3 Длительность: 921.6 сек
Эмоция: neutral
Файл: fetisova_inna_out_2023_09_26-15_55_10_79273659168_2kwp.mp3 Длительность: 461.6 сек
Эмоция: neutral
Файл: mikhaylova_polina_out_2023_09_29-10_16_35_79095082928_p4j6.mp3 Длительность: 580.2 сек
Эмоция: neutral
Файл: 79260007055_in_2023_09_26-12_32_39_79266662497_ko7b.mp3 Длительность: 450.6 сек
Эмоция: neutral
Файл: morozova_anastasia_out_2023_09_12-11_04_57_79255070348_7mfb.mp3 Длительность: 450.5 сек
Эмоция: neutral
Файл: 79260102604_in_2023_09_28-12_43_42_79031411293_do7q.mp3 Длительность: 849.7 сек
Эмоция: neutral
Файл: 79260102604_in_2023_09_18-15_58_35_79267063464_3nwl.mp3 Длительность: 558.5 сек
Эмоция: positive
Файл: 74951824000_in_2023_09_27-14_19_24_79041834531_nqnw.mp3 Длительность: 451.8 сек
Эмоция: neutral
Файл: dokuz_nina_out_2023_09_27-11_16_19_79296431313_3w5j.mp3 Длительность: 332.4 сек
Эмоция: neutral
Файл: 79266440512_in_2023_09_26-16_33_00_79629379315_wdcw.mp3 Д

In [10]:
df_preds.emotion.value_counts()

emotion
neutral     17
positive     2
Name: count, dtype: int64

In [11]:
df_preds.emotion.value_counts(normalize=True)

emotion
neutral     0.894737
positive    0.105263
Name: proportion, dtype: float64

In [12]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(f'{WORK_PATH}/*.csv')
zip_filename = WORK_PATH.joinpath('predictions.zip')
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)

./df_preds_rings_29_9.csv


/kaggle/working/predictions.zip